MusicLM - using LucidRain's pytorch implementation. 

In [ ]:
!pip install musiclm-pytorch

In [ ]:
# !pip install audiolm-pytorch this mgiht be needed idk

In [ ]:
!pip install encodec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from musiclm_pytorch import MuLaN, AudioSpectrogramTransformer, TextTransformer
from encodec import EncodecModel
from encodec.utils import convert_audio
from PIL import Image
import numpy as np
import torchaudio
import torch
from IPython.display import Audio
import torchvision.transforms as transforms  
import matplotlib.pyplot as plt

# define all dataset paths, checkpoints, etc
dataset_folder = "placeholder_dataset"
soundstream_ckpt = "results/soundstream.8.pt" # this can change depending on number of steps
hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = f'hubert/hubert_base_ls960_L9_km500.bin' # listed in row "HuBERT Base (~95M params)", column Quantizer

In [ ]:
def get_sinewave(freq=440.0, duration_ms=200, volume=1.0, sample_rate=44100.0):
  # code adapted from https://stackoverflow.com/a/33913403
  audio = []
  num_samples = duration_ms * (sample_rate / 1000.0)
  for x in range(int(num_samples)):
    audio.append(volume * math.sin(2 * math.pi * freq * (x / sample_rate)))
  return audio

In [ ]:
# Placeholder data generation
def save_wav(file_name, audio, sample_rate=44100.0):
  # Open up a wav file
  wav_file=wave.open(file_name,"w")
  # wav params
  nchannels = 1
  sampwidth = 2
  # 44100 is the industry standard sample rate - CD quality.  If you need to
  # save on file size you can adjust it downwards. The stanard for low quality
  # is 8000 or 8kHz.
  nframes = len(audio)
  comptype = "NONE"
  compname = "not compressed"
  wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))
  # WAV files here are using short, 16 bit, signed integers for the 
  # sample size.  So we multiply the floating point data we have by 32767, the
  # maximum value for a short integer.  NOTE: It is theortically possible to
  # use the floating point -1.0 to 1.0 data directly in a WAV file but not
  # obvious how to do that using the wave module in python.
  for sample in audio:
      wav_file.writeframes(struct.pack('h', int( sample * 32767.0 ))) # Sort this out, this is needless
  wav_file.close()
  return

def make_placeholder_dataset():
  # Make a placeholder dataset with a few .wav files that you can "train" on, just to verify things work e2e
  if os.path.isdir(dataset_folder):
    return
  os.makedirs(dataset_folder)
  save_wav(f"{dataset_folder}/example.wav", get_sinewave())
  save_wav(f"{dataset_folder}/example2.wav", get_sinewave(duration_ms=500))
  os.makedirs(f"{dataset_folder}/subdirectory")
  save_wav(f"{dataset_folder}/subdirectory/example.wav", get_sinewave(freq=330.0)) # take this line out and sine wave thing for real training attempt

make_placeholder_dataset()

#Train MuLan:

This part isn't finished, need to replace wavs and then I think it's done. 

In [ ]:
audio_transformer = AudioSpectrogramTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    spec_n_fft = 128,
    spec_win_length = 24,
    spec_aug_stretch_factor = 0.8
)

text_transformer = TextTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64
)

mulan = MuLaN(
    audio_transformer = audio_transformer,
    text_transformer = text_transformer
)

# get a ton of <sound, text> pairs and train

wavs = torch.randn(2, 1024)
texts = torch.randint(0, 20000, (2, 256))

loss = mulan(wavs, texts)
loss.backward()

# after much training, you can embed sounds and text into a joint embedding space
# for conditioning the audio LM

In [ ]:
embeds = mulan.get_audio_latents(wavs)  # during training ?? what does this mean, during training vs inference?

embeds = mulan.get_text_latents(texts)  # during inference, I mean do these bits even need to run, this embeds would overwrite the one above

#To do data loader. 

But what does this mean? 

In [ ]:
ds_path = dataset_folder

In [3]:
wavs = torch.randn(2, 1024)
texts = torch.randint(0, 20000, (2, 256))

In [4]:
wavs.shape, texts.shape

(torch.Size([2, 1024]), torch.Size([2, 256]))

In [ ]:
from musiclm_pytorch import MuLaNEmbedQuantizer
# setup the quantizer with the namespaced conditioning embeddings, unique per quantizer as well as namespace (per transformer)
quantizer = MuLaNEmbedQuantizer(
    mulan = mulan,                          # pass in trained mulan from above
    conditioning_dims = (1024, 1024, 1024), # say all three transformers have model dimensions of 1024
    namespaces = ('semantic', 'coarse', 'fine')
)
# now say you want the conditioning embeddings for semantic transformer
wavs = torch.randn(2, 1024)
conds = quantizer(wavs = wavs, namespace = 'semantic') # (2, 8, 1024) - 8 is number of quantizers, dunno what the two is about, stereo audio?

#Semantic Transformer:

In [ ]:
from audiolm_pytorch import HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer

In [ ]:
# hubert checkpoints can be downloaded at
# https://github.com/facebookresearch/fairseq/tree/main/examples/hubert
if not os.path.isdir("hubert"):
  os.makedirs("hubert")
if not os.path.isfile(hubert_ckpt):
  hubert_ckpt_download = f"https://dl.fbaipublicfiles.com/{hubert_ckpt}"
  urllib.request.urlretrieve(hubert_ckpt_download, f"./{hubert_ckpt}")
if not os.path.isfile(hubert_quantizer):
  hubert_quantizer_download = f"https://dl.fbaipublicfiles.com/{hubert_quantizer}"
  urllib.request.urlretrieve(hubert_quantizer_download, f"./{hubert_quantizer}")

# work out if we need to be getting or dl the checkpoints if not already there
wav2vec = HubertWithKmeans(
    checkpoint_path = './hubert/hubert_base_ls960.pt',
    kmeans_path = './hubert/hubert_base_ls960_L9_km500.bin'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6,
    audio_text_condition = True      # this must be set to True (same for CoarseTransformer and FineTransformers)
).cuda()

trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1000 # idk the number of training steps needed here for the semantic transformer
)

trainer.train()

# Coarse Transformer:

In [ ]:
from audiolm_pytorch import CoarseTransformer, CoarseTransformerWrapper, CoarseTransformerTrainer, FineTransformer, FineTransformerWrapper, FineTransformerTrainer, AudioLM

In [ ]:
from audiolm_pytorch import EncodecWrapper

In [ ]:
wav2vec = HubertWithKmeans(
    checkpoint_path = f'./{hubert_ckpt}',
    kmeans_path = f'./{hubert_quantizer}'
)

# Instantiate a pretrained EnCodec model
encodec = EncodecWrapper(
        target_sample_hz = 24000,
        strides = (2, 4, 5, 8),
        num_quantizers = 8,
)

coarse_transformer = CoarseTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    codebook_size = 1024,
    num_coarse_quantizers = 3,
    dim = 512,
    depth = 6,
    audio_text_condition = True # adding audio text cond true    
)

trainer = CoarseTransformerTrainer(
    transformer = coarse_transformer,
    codec = encodec, # change to encodec
    wav2vec = wav2vec,
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    save_results_every = 1000,
    save_model_every = 1000,
    num_train_steps = 10000,
    
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason

trainer.train()

#Fine transformer

In [ ]:
encodec = EncodecWrapper(
        target_sample_hz = 24000,
        strides = (2, 4, 5, 8),
        num_quantizers = 8,
)

fine_transformer = FineTransformer(
    num_coarse_quantizers = 3, # could test 2 by 2 if using n_q = 4, or 1 by 3
    num_fine_quantizers = 5,
    codebook_size = 1024,
    dim = 512,
    depth = 6,
    audio_text_condition = True   # adding audio text cond true
)

trainer = FineTransformerTrainer(
    transformer = fine_transformer,
    codec = encodec, # change to encodec
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 10000,
    save_model_every = 1000
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason
trainer.train()

#Inference:

In [ ]:
encodec = EncodecWrapper(
        target_sample_hz = 24000,
        strides = (2, 4, 5, 8),
        num_quantizers = 8,
)

In [ ]:
# Everything together
audiolm = AudioLM(
    wav2vec = wav2vec,
    codec = encodec,
    semantic_transformer = semantic_transformer,
    coarse_transformer = coarse_transformer,
    fine_transformer = fine_transformer
)

generated_wav = audiolm(batch_size = 1)

In [ ]:
# you need the trained AudioLM (audio_lm) from above
# with the MulanEmbedQuantizer (mulan_embed_quantizer)
from musiclm_pytorch import MusicLM

musiclm = MusicLM(
    audio_lm = audio_lm, # ? do we need (batch_size = 1) here as well?
    mulan_embed_quantizer = mulan_embed_quantizer
)

music = musiclm('the crystalline sounds of the piano in a ballroom', num_samples = 4) # sample 4 and pick the top match with mulan